# Loop 25 Analysis: Asymmetric Improvement Strategy

**Key Insight from Research:**
- Top teams reach sub-69 scores by exploiting ASYMMETRY
- Asymmetric layouts yield better scores especially for N < 60
- The 70.6 to 68.9 gap (2.4%) requires targeted per-N improvements

**Goal:** Identify specific N values where asymmetric layouts could improve over baseline

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import matplotlib.pyplot as plt

TREE_TEMPLATE = [
    (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5), (0.2, 0.25), (0.1, 0.25),
    (0.35, 0.0), (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2), (-0.075, 0.0),
    (-0.35, 0.0), (-0.1, 0.25), (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5)
]

def parse_s_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def create_tree_polygon(x, y, angle):
    tree = Polygon(TREE_TEMPLATE)
    tree = rotate(tree, angle, origin=(0, 0), use_radians=False)
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(trees):
    all_x, all_y = [], []
    for tree in trees:
        minx, miny, maxx, maxy = tree.bounds
        all_x.extend([minx, maxx])
        all_y.extend([miny, maxy])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

# Load baseline
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].apply(parse_s_value)
df['y'] = df['y'].apply(parse_s_value)
df['deg'] = df['deg'].apply(parse_s_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

print(f"Loaded {len(df)} rows")
print(f"N values: {df['n'].min()} to {df['n'].max()}")
print(f"Total trees: {len(df)}")
print(f"Expected: {sum(range(1, 201))} = {200*201//2}")
print()

In [ ]:
# Calculate per-N scores and analyze symmetry
per_n_data = {}

for n in range(1, 201):
    group = df[df['n'] == n]
    trees = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    score = (side ** 2) / n
    
    # Get angles and positions
    angles = group['deg'].values
    xs = group['x'].values
    ys = group['y'].values
    
    # Analyze angle distribution
    angles_mod = angles % 360
    unique_angles = len(np.unique(np.round(angles_mod, 1)))
    
    # Check for symmetry patterns
    # Rotational symmetry: angles differ by 360/k for some k
    angle_diffs = np.diff(np.sort(angles_mod))
    
    per_n_data[n] = {
        'score': score,
        'side': side,
        'unique_angles': unique_angles,
        'angle_std': np.std(angles_mod),
        'x_range': xs.max() - xs.min(),
        'y_range': ys.max() - ys.min(),
        'angles': angles_mod
    }

print("Per-N analysis complete")
print(f"Total score: {sum(d['score'] for d in per_n_data.values()):.6f}")
print(f"Target: 68.919154")
print(f"Gap: {sum(d['score'] for d in per_n_data.values()) - 68.919154:.6f}")

In [ ]:
# Identify N values with highest improvement potential
# Focus on N < 60 where asymmetry helps most (per research)

print("\n=== N < 60 Analysis (Asymmetry helps most here) ===")
print("="*70)

small_n_scores = [(n, per_n_data[n]['score']) for n in range(1, 60)]
small_n_scores.sort(key=lambda x: x[1], reverse=True)

print("\nTop 20 worst-performing N values (N < 60):")
for n, score in small_n_scores[:20]:
    data = per_n_data[n]
    print(f"  N={n:3d}: score={score:.6f}, unique_angles={data['unique_angles']:3d}, angle_std={data['angle_std']:.1f}")

print("\n=== Score contribution by N range ===")
ranges = [(1, 10), (11, 20), (21, 30), (31, 40), (41, 50), (51, 60), (61, 100), (101, 150), (151, 200)]
for start, end in ranges:
    total = sum(per_n_data[n]['score'] for n in range(start, end+1))
    count = end - start + 1
    avg = total / count
    print(f"  N={start:3d}-{end:3d}: total={total:.4f}, avg={avg:.4f}, count={count}")

print(f"\nTotal score: {sum(d['score'] for d in per_n_data.values()):.6f}")

In [ ]:
# Visualize a few N values to understand the structure
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

test_ns = [5, 10, 20, 30, 50, 100]

for ax, n in zip(axes.flat, test_ns):
    group = df[df['n'] == n]
    
    for _, row in group.iterrows():
        tree = create_tree_polygon(row['x'], row['y'], row['deg'])
        x, y = tree.exterior.xy
        ax.fill(x, y, alpha=0.5, edgecolor='green', linewidth=0.5)
    
    ax.set_aspect('equal')
    ax.set_title(f'N={n}, score={per_n_data[n]["score"]:.4f}')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/code/exploration/baseline_visualization.png', dpi=100)
plt.show()
print("Saved visualization to /home/code/exploration/baseline_visualization.png")

In [ ]:
# Key insight: Try to find N values where a small perturbation could help
# Focus on N values where the baseline has high angle diversity (not tessellation)

print("\n=== Identifying N values with improvement potential ===")
print("="*70)

# N values with high angle diversity (not simple tessellation)
high_diversity = [(n, per_n_data[n]) for n in range(1, 60) 
                  if per_n_data[n]['unique_angles'] > n * 0.5]

print(f"\nN values with high angle diversity (unique_angles > n/2):")
for n, data in sorted(high_diversity, key=lambda x: x[1]['score'], reverse=True)[:15]:
    print(f"  N={n:3d}: score={data['score']:.6f}, unique_angles={data['unique_angles']}, angle_std={data['angle_std']:.1f}")

# Calculate theoretical lower bound (if trees could be packed perfectly)
print("\n=== Theoretical Analysis ===")
print("Single tree area (approx): 0.35 * 1.0 = 0.35 sq units")
print("But actual tree polygon area is smaller due to shape")

# Calculate actual tree area
tree = Polygon(TREE_TEMPLATE)
print(f"Actual tree polygon area: {tree.area:.6f} sq units")

for n in [10, 20, 50, 100, 200]:
    min_area = n * tree.area
    min_side = np.sqrt(min_area)
    theoretical_score = min_area / n  # = tree.area
    actual_score = per_n_data[n]['score']
    efficiency = tree.area / actual_score * 100
    print(f"N={n:3d}: theoretical_min_side={min_side:.4f}, actual_side={per_n_data[n]['side']:.4f}, efficiency={efficiency:.1f}%")

In [ ]:
# Try a simple asymmetric perturbation on a specific N value
# Start with N=10 as a test case

from shapely.ops import unary_union
import random

def check_overlap(tree1, tree2):
    return tree1.overlaps(tree2) or tree1.contains(tree2) or tree2.contains(tree1)

def has_any_overlap(trees):
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if check_overlap(trees[i], trees[j]):
                return True
    return False

def local_search_asymmetric(n, max_iter=5000, verbose=False):
    """Try to improve N by local search with asymmetric perturbations."""
    group = df[df['n'] == n].copy()
    
    # Get current configuration
    xs = group['x'].values.copy()
    ys = group['y'].values.copy()
    angles = group['deg'].values.copy()
    
    trees = [create_tree_polygon(xs[i], ys[i], angles[i]) for i in range(n)]
    current_side = get_bounding_box_side(trees)
    best_side = current_side
    best_config = (xs.copy(), ys.copy(), angles.copy())
    
    # Local search
    for iteration in range(max_iter):
        # Pick a random tree
        i = random.randint(0, n-1)
        
        # Try a small perturbation
        dx = random.gauss(0, 0.02)
        dy = random.gauss(0, 0.02)
        da = random.gauss(0, 5)
        
        new_x = xs[i] + dx
        new_y = ys[i] + dy
        new_angle = angles[i] + da
        
        # Create new tree
        new_tree = create_tree_polygon(new_x, new_y, new_angle)
        
        # Check overlap with other trees
        has_overlap = False
        for j in range(n):
            if j != i and check_overlap(new_tree, trees[j]):
                has_overlap = True
                break
        
        if not has_overlap:
            # Update configuration
            old_tree = trees[i]
            trees[i] = new_tree
            new_side = get_bounding_box_side(trees)
            
            if new_side < current_side:
                xs[i] = new_x
                ys[i] = new_y
                angles[i] = new_angle
                current_side = new_side
                
                if current_side < best_side:
                    best_side = current_side
                    best_config = (xs.copy(), ys.copy(), angles.copy())
                    if verbose:
                        print(f"  Iter {iteration}: improved to {best_side:.6f}")
            else:
                trees[i] = old_tree
    
    return best_side, best_config

print("Testing local search on N=10...")
baseline_score = per_n_data[10]['score']
print(f"Baseline N=10 score: {baseline_score:.6f}")

best_side, best_config = local_search_asymmetric(10, max_iter=10000, verbose=True)
new_score = (best_side ** 2) / 10
print(f"\nAfter local search: {new_score:.6f}")
print(f"Improvement: {baseline_score - new_score:.6f}")

In [ ]:
# Test on multiple N values to find where improvements are possible
print("\n=== Testing local search on N=1-30 ===")
print("="*70)

improvements = []

for n in range(1, 31):
    baseline_score = per_n_data[n]['score']
    best_side, _ = local_search_asymmetric(n, max_iter=5000)
    new_score = (best_side ** 2) / n
    improvement = baseline_score - new_score
    improvements.append((n, baseline_score, new_score, improvement))
    
    if improvement > 1e-6:
        print(f"N={n:3d}: baseline={baseline_score:.6f} -> {new_score:.6f}, improvement={improvement:.6f} ✓")
    else:
        print(f"N={n:3d}: baseline={baseline_score:.6f} -> {new_score:.6f}, no improvement")

print("\n=== Summary ===")
total_improvement = sum(imp[3] for imp in improvements if imp[3] > 0)
print(f"Total improvement from N=1-30: {total_improvement:.6f}")
print(f"N values improved: {sum(1 for imp in improvements if imp[3] > 1e-6)}")
print(f"\nThis is {'significant' if total_improvement > 0.01 else 'NOT significant'} compared to gap of 1.708")